## W2-CRISPR-cBEST workflow


Base-editing with CRISPR-BEST is a DSB-free method that uses a single sgRNA to target a specific genomic location with single-nucleotide–resolution(Figure 3A) which can enable in-vivo protein engineering or as it is often used, for introducing early stop-codons10,12. It can be used to introduce targeted point mutations in the genome of Streptomyces, facilitating studies on gene function and protein engineering. To get started quickly users can download the pCRISPR-cBEST plasmid file, the genome file for S.coelicor (A3) and streptoAIM will generate your plasmids based on the genes you chose to target (Figure 3, S5). 

In [1]:
import sys
import os

# Ensure the src directory is in the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import os
import pandas as pd
from pydna.dseqrecord import Dseqrecord
from datetime import datetime

from streptocad.sequence_loading.sequence_loading import  load_and_process_plasmid, load_and_process_genome_sequences, annotate_dseqrecord, check_and_convert_input, process_specified_gene_sequences_from_record
from streptocad.utils import polymerase_dict,ProjectDirectory, extract_metadata_to_dataframe
from streptocad.crispr.guideRNAcas3_9_12 import extract_sgRNAs, SgRNAargs
from streptocad.cloning.ssDNA_bridging import assemble_plasmids_by_ssDNA_bridging, make_ssDNA_oligos
from streptocad.crispr.crispr_best import identify_base_editing_sites, filter_sgrnas_for_base_editing, process_base_editing
from streptocad.cloning.plasmid_processing import annotate_plasmid_with_sgrnas
from streptocad.primers.primer_generation import find_best_check_primers_from_genome, create_idt_order_dataframe, primers_to_IDT

## INPUT

In [2]:
# Inputs
# 1 Add genome of choice (genbank, fasta)
path_to_genome = '../../data/genomes/Streptomyces_coelicolor_A3_chromosome.gb'
genome = load_and_process_genome_sequences(path_to_genome)[0]

# 2 Add plasmid 
path_to_plasmid = '../../data/plasmids/ pCRISPR-cBEST Sequences.gbk'
clean_plasmid = load_and_process_plasmid(path_to_plasmid)

# 3 Choose genes to knock out (list)
genes_to_KO = ['SCO5087']#,'SCO5088', 'SCO5089', 'SCO5090', 'SCO5091', 'SCO5092', 'SCO5093']
#genes_to_KO = ['80000-100000', '4000-7000', '9000-14000','15000-20000']


#### Advanced settings ####
# 3 Choose polymerase and target melting temperature FOR CHECKING PRIMERS
chosen_polymerase = polymerase_dict['Phusion High-Fidelity DNA Polymerase (GC Buffer)']

melting_temperature = 65
primer_concentration = 0.4 
primer_number_increment = 1
flanking_region_number = 500

# 4 Filtering metrics for sgRNAs
gc_upper = 0.99
gc_lower = 0.01
off_target_seed = 13
off_target_upper = 10
cas_type='cas9'
number_of_sgRNAs_per_group = 5
only_stop_codons = True
flanking_region = 500

# 6 Choose overlapping sequences for our plasmid we can use the following
#As per the article **"CRISPR–Cas9, CRISPRi and CRISPR-BEST-mediated genetic manipulation in streptomycetes"** we need the following oligoes: 
#CGGTTGGTAGGATCGACGGC **-N20-** GTTTTAGAGCTAGAAATAGC
up_homology = Dseqrecord('cggttggtaggatcgacggc') # V1 homology : CGGTTGGTAGGATCGACGGC
dw_homology = Dseqrecord('tatgtcctccgagaccggcc') # V1 homology : GTTTTAGAGCTAGAAATAGC

# Computation

In [3]:
print(clean_plasmid.id)


pCRISPR-cBEST(#125689)


In [4]:
from teemi.design.fetch_sequences import read_genbank_files     
sequences = read_genbank_files(path_to_plasmid)
sequences


[SeqRecord(seq=Seq('GGCCGTCGACGACGACGCGTCCGCCTGCCTGCTTTTGGCCGGCGGGCCGGGCCG...CCA'), id='pCRISPR-cBEST(#125689)', name='pCRISPR-cBEST', description='C to T base editor for actinomycetes', dbxrefs=[])]

In [5]:
len(genome.features)


25824

In [6]:
target_dict, genes_to_KO, annotation_input = check_and_convert_input(genes_to_KO)

print(annotation_input)
if annotation_input == True:
    genome = annotate_dseqrecord(genome, target_dict)


len(genome.features)

False


25824

In [7]:
genes_to_KO


['SCO5087']

We need to check the input genes so we can find sgRNAs. 

In [8]:

# Initialize SgRNAargs with desired parameters
args = SgRNAargs(genome, 
                genes_to_KO,
                step=['find', 'filter'],
                gc_upper = gc_upper,
                gc_lower = gc_lower,
                off_target_seed = off_target_seed,
                off_target_upper = off_target_upper,
                cas_type='cas9',
                )

sgrna_df = extract_sgRNAs(args)
sgrna_df

Pam was found outside designated locus_tag: SCO5087. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO5087. To incorporate this extent borders. Skipping to next locus tag.
Pam was found outside designated locus_tag: SCO5087. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO5087. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO5087. To incorporate this extent borders. Skipping to next locus tag.


,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
0,NC_003888.3,SCO5087,5529801,1,-1,26,0.80,CGG,TCCACCGGCGCCGCGTCCAG,GCGCCGCGTCCAG,0
161,NC_003888.3,SCO5087,5529801,1,1,1189,0.80,TGG,GCTGGGCGCGATCGGCTCGC,GCGATCGGCTCGC,0
162,NC_003888.3,SCO5087,5529801,1,1,1181,0.75,CGG,GGCCACTCGCTGGGCGCGAT,CGCTGGGCGCGAT,0
163,NC_003888.3,SCO5087,5529801,1,1,1172,0.65,GGG,TCGATGGTCGGCCACTCGCT,TCGGCCACTCGCT,0
165,NC_003888.3,SCO5087,5529801,1,1,1160,0.50,CGG,AGCTCGATCAAGTCGATGGT,TCAAGTCGATGGT,0
...,...,...,...,...,...,...,...,...,...,...,...
231,NC_003888.3,SCO5087,5529801,1,1,469,0.80,TGG,GGTGGCAGCGGCGACCAGCC,GCGGCGACCAGCC,5
42,NC_003888.3,SCO5087,5529801,1,-1,424,0.80,GGG,GGGCGGGGTCGAGGGTGTCC,GTCGAGGGTGTCC,5
39,NC_003888.3,SCO5087,5529801,1,-1,394,0.80,CGG,CGCTGGCGGCGAACGCCTCC,GGCGAACGCCTCC,6
256,NC_003888.3,SCO5087,5529801,1,1,307,0.75,AGG,CGACTTCGACCCGGTCGCCG,GACCCGGTCGCCG,7


In [9]:
value_counts = sgrna_df['locus_tag'].value_counts()
value_counts

locus_tag
SCO5087    289
Name: count, dtype: int64

In [10]:
len(genome.features)

25824

In [11]:
# Load gene sequences from the genome
gene_sequences = process_specified_gene_sequences_from_record(genome, genes_to_KO)
genes_to_KO_dict = {locus_tag: gene_sequences[locus_tag] for locus_tag in genes_to_KO if locus_tag in gene_sequences}
print(genes_to_KO_dict)
# Identify and annotate base editing sites
sgrna_df_with_editing = identify_base_editing_sites(sgrna_df)

# filter out only sgRNAs that result in base-editing
filtered_sgrna_df_for_base_editing = filter_sgrnas_for_base_editing(sgrna_df_with_editing)
filtered_sgrna_df_for_base_editing

{'SCO5087': Dseq(-1404)
GTGC..ATGA
CACG..TACT}


,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines
0,NC_003888.3,SCO5087,5529801,1,-1,26,0.80,CGG,TCCACCGGCGCCGCGTCCAG,GCGCCGCGTCCAG,0,"0,0,0,1","3,5,6,9"
161,NC_003888.3,SCO5087,5529801,1,1,1189,0.80,TGG,GCTGGGCGCGATCGGCTCGC,GCGATCGGCTCGC,0,"1,1","7,9"
162,NC_003888.3,SCO5087,5529801,1,1,1181,0.75,CGG,GGCCACTCGCTGGGCGCGAT,CGCTGGGCGCGAT,0,"1,0,0,0,1","3,4,6,8,10"
163,NC_003888.3,SCO5087,5529801,1,1,1172,0.65,GGG,TCGATGGTCGGCCACTCGCT,TCGGCCACTCGCT,0,0,9
165,NC_003888.3,SCO5087,5529801,1,1,1160,0.50,CGG,AGCTCGATCAAGTCGATGGT,TCAAGTCGATGGT,0,"1,0,0","3,5,9"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,NC_003888.3,SCO5087,5529801,1,1,469,0.80,TGG,GGTGGCAGCGGCGACCAGCC,GCGGCGACCAGCC,5,"1,1","6,9"
42,NC_003888.3,SCO5087,5529801,1,-1,424,0.80,GGG,GGGCGGGGTCGAGGGTGTCC,GTCGAGGGTGTCC,5,"1,0","4,10"
39,NC_003888.3,SCO5087,5529801,1,-1,394,0.80,CGG,CGCTGGCGGCGAACGCCTCC,GGCGAACGCCTCC,6,"1,1,1","3,7,10"
256,NC_003888.3,SCO5087,5529801,1,1,307,0.75,AGG,CGACTTCGACCCGGTCGCCG,GACCCGGTCGCCG,7,"0,0,0","4,7,10"


In [12]:
# Process the DataFrame to apply C-to-T mutations
mutated_sgrna_df = process_base_editing(filtered_sgrna_df_for_base_editing, 
                                        genes_to_KO_dict, 
                                        only_stop_codons = False, 
                                        editing_context= False)
mutated_sgrna_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
0,NC_003888.3,SCO5087,5529801,1,-1,26,0.80,CGG,TCCACCGGCGCCGCGTCCAG,GCGCCGCGTCCAG,0,"0,0,0,1","3,5,6,9",V8I
161,NC_003888.3,SCO5087,5529801,1,1,1189,0.80,TGG,GCTGGGCGCGATCGGCTCGC,GCGATCGGCTCGC,0,"1,1","7,9",A393V
162,NC_003888.3,SCO5087,5529801,1,1,1181,0.75,CGG,GGCCACTCGCTGGGCGCGAT,CGCTGGGCGCGAT,0,"1,0,0,0,1","3,4,6,8,10","H389Y, S390L"
165,NC_003888.3,SCO5087,5529801,1,1,1160,0.50,CGG,AGCTCGATCAAGTCGATGGT,TCAAGTCGATGGT,0,"1,0,0","3,5,9",S382L
167,NC_003888.3,SCO5087,5529801,1,1,1135,0.75,CGG,CGAACACGCCCGGCGCACTC,GCCCGGCGCACTC,0,"0,0,1,0","5,7,9,10","H374Y, A375V"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,NC_003888.3,SCO5087,5529801,1,-1,1018,0.75,CGG,GGGACTCGTCGAGGGCGACC,GTCGAGGGCGACC,4,"0,0,0","5,7,10","D337N, E338K"
231,NC_003888.3,SCO5087,5529801,1,1,469,0.80,TGG,GGTGGCAGCGGCGACCAGCC,GCGGCGACCAGCC,5,"1,1","6,9","A152V, A153V"
42,NC_003888.3,SCO5087,5529801,1,-1,424,0.80,GGG,GGGCGGGGTCGAGGGTGTCC,GTCGAGGGTGTCC,5,"1,0","4,10","D139N, A141T"
39,NC_003888.3,SCO5087,5529801,1,-1,394,0.80,CGG,CGCTGGCGGCGAACGCCTCC,GGCGAACGCCTCC,6,"1,1,1","3,7,10","A129T, A130T, S131N"


In [13]:
# Process the DataFrame to apply C-to-T mutations
mutated_sgrna_df = process_base_editing(filtered_sgrna_df_for_base_editing, 
                                        genes_to_KO_dict, 
                                        only_stop_codons = False, 
                                        editing_context= True)
mutated_sgrna_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
180,NC_003888.3,SCO5087,5529801,1,1,964,0.60,AGG,GTACCACATGACGGGCCTGA,ATGACGGGCCTGA,0,"0,0,0","4,5,7",H317Y
157,NC_003888.3,SCO5087,5529801,1,1,1291,0.60,TGG,CGATCTCGACTACGTTCCGC,GACTACGTTCCGC,0,"0,0,0","5,7,10",L426F
130,NC_003888.3,SCO5087,5529801,1,-1,1270,0.75,AGG,ACTCGGGGTCGCTGGTGCGC,GTCGCTGGTGCGC,0,"0,0","4,10","D421N, E423K"
132,NC_003888.3,SCO5087,5529801,1,-1,1285,0.60,GGG,CGTAGTCGAGATCGCACTCG,GAGATCGCACTCG,0,0,7,D427N
133,NC_003888.3,SCO5087,5529801,1,-1,1286,0.55,GGG,ACGTAGTCGAGATCGCACTC,CGAGATCGCACTC,0,0,8,D427N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,NC_003888.3,SCO5087,5529801,1,-1,1038,0.70,GGG,GATGTCCGTCGCGTCCGTGC,GTCGCGTCCGTGC,2,"0,0,0","6,7,10",D345N
254,NC_003888.3,SCO5087,5529801,1,1,314,0.75,CGG,GACCCGGTCGCCGAGGGCTT,TCGCCGAGGGCTT,2,"0,0,0,0","3,4,5,9",P100L
255,NC_003888.3,SCO5087,5529801,1,1,308,0.70,GGG,GACTTCGACCCGGTCGCCGA,ACCCGGTCGCCGA,2,"0,0,0,0","3,6,9,10",P100S
78,NC_003888.3,SCO5087,5529801,1,-1,745,0.65,GGG,CGACGACGATCGGGGTGATC,GATCGGGGTGATC,2,"0,0","4,7","V247I, V248I"


In [14]:
# Process the DataFrame to apply C-to-T mutations
mutated_sgrna_df = process_base_editing(filtered_sgrna_df_for_base_editing, 
                                        genes_to_KO_dict, 
                                        only_stop_codons = True, 
                                        editing_context= True)
mutated_sgrna_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
258,NC_003888.3,SCO5087,5529801,1,1,283,0.75,AGG,CCGTTCACAGGTCGCGGCGG,CAGGTCGCGGCGG,0,"0,0","6,8","S90L, Q91*"
54,NC_003888.3,SCO5087,5529801,1,-1,522,0.70,CGG,GTCGACCTCCCAGTCACGGC,TCCCAGTCACGGC,0,"0,0,0,0,0","3,6,7,9,10","W171*, E172K, V173I, D174N"


In [15]:
# Filter the DataFrame to retain only up to 5 sgRNA sequences per locus_tag
filtered_df = mutated_sgrna_df.groupby('locus_tag').head(number_of_sgRNAs_per_group)
filtered_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
258,NC_003888.3,SCO5087,5529801,1,1,283,0.75,AGG,CCGTTCACAGGTCGCGGCGG,CAGGTCGCGGCGG,0,"0,0","6,8","S90L, Q91*"
54,NC_003888.3,SCO5087,5529801,1,-1,522,0.70,CGG,GTCGACCTCCCAGTCACGGC,TCCCAGTCACGGC,0,"0,0,0,0,0","3,6,7,9,10","W171*, E172K, V173I, D174N"


## Output

In [16]:
# MAke oligoes
list_of_ssDNAs = make_ssDNA_oligos(filtered_df, upstream_ovh = up_homology,
                      downstream_ovh=dw_homology)


# cut plasmid
from Bio.Restriction import NcoI, NheI
linearized_plasmid = sorted(clean_plasmid.cut(NcoI,NheI ), key=lambda x: len(x), reverse=True)[0]

# assemble plasmid
sgRNA_vectors = assemble_plasmids_by_ssDNA_bridging(list_of_ssDNAs,linearized_plasmid)

In [17]:
# Constructing a meaningful name, ID, and description for the assembled plasmid using user input
targeting_info = []
for index, row in filtered_df.iterrows():
    formatted_str = f"pCRISPR-BEST_{row['locus_tag']}_p{row['sgrna_loc']}"
    targeting_info.append(formatted_str)

for i in range(len(sgRNA_vectors)):
    sgRNA_vectors[i].name = f'{targeting_info[i]}_#{i+1}'
    sgRNA_vectors[i].id = sgRNA_vectors[i].name  # Using the same value for ID as for name for simplicity
    sgRNA_vectors[i].description = f'Assembled plasmid targeting {", ".join(genes_to_KO)} for base-editing, assembled using StreptoCAD.'

# annotate plasmids
for plasmid in sgRNA_vectors: 
    annotate_plasmid_with_sgrnas(plasmid, filtered_df)

# PRINT
print_plasmids = False
if print_plasmids: 
    for vector in sgRNA_vectors: 
        vector.write(f"../../data/plasmids/sgRNA_plasmids_pCRISPR_cBEST/{vector.id}.gb")

In [18]:
integration_names = filtered_df.apply(lambda row: f"sgRNA_{row['locus_tag']}({row['sgrna_loc']})", axis=1).tolist()
plasmid_metadata_df = extract_metadata_to_dataframe(sgRNA_vectors,
                                                    clean_plasmid,
                                                    integration_names)

plasmid_metadata_df

,plasmid_name,date,original_plasmid,integration,size
0,pCRISPR-BEST_SCO5087_p283_#1,2025-02-25,pCRISPR-cBEST(#125689),sgRNA_SCO5087(283),11817
1,pCRISPR-BEST_SCO5087_p522_#2,2025-02-25,pCRISPR-cBEST(#125689),sgRNA_SCO5087(522),11817


# IDT primers

In [19]:
idt_df1 = primers_to_IDT(list_of_ssDNAs)
idt_df1

,Name,Sequence,Concentration,Purification
0,SCO5087_loc_283,cggttggtaggatcgacggcCCGTTCACAGGTCGCGGCGGtatgtc...,25nm,STD
1,SCO5087_loc_522,cggttggtaggatcgacggcGTCGACCTCCCAGTCACGGCtatgtc...,25nm,STD


In [20]:
genes_to_KO

['SCO5087']

In [21]:
# Getting checking primers
checking_primers_df = find_best_check_primers_from_genome(genome, 
                                       genes_to_KO, 
                                       flanking_region=flanking_region,
                                       target_tm = melting_temperature, 
                                        primer_concentration = primer_concentration, 
                                        polymerase = chosen_polymerase)
checking_primers_df.to_clipboard()

In [22]:
checking_primers_df

,locus tag,f_primer_name,r_primer_name,f_primer_sequences(5-3),r_primer_sequences(5-3),f_tm,r_tm,ta,flanking_region,annealing_temperature,...,homodimer_reverse_tm,homodimer_reverse_deltaG (kcal/mol),heterodimer_tm,heterodimer_deltaG (kcal/mol),hairpin_forward_structure_found,hairpin_forward_tm,hairpin_forward_deltaG (kcal/mol),hairpin_reverse_structure_found,hairpin_reverse_tm,hairpin_reverse_deltaG (kcal/mol)
0,SCO5087,SCO5087_fwd_checking_primer,SCO5087_rev_checking_primer,GACGATTCGGCCCGTG,CAGGGCGTCCAGGC,59,58,61,500,61,...,9.706246,-3.415427,1.918567,-2.396641,False,0.0,0.0,False,0.0,0.0


In [23]:
idt_df2 = create_idt_order_dataframe(checking_primers_df)
idt_df2

,Name,Sequence,Concentration,Purification
0,SCO5087_fwd_checking_primer,GACGATTCGGCCCGTG,25nm,STD
1,SCO5087_rev_checking_primer,CAGGGCGTCCAGGC,25nm,STD


In [24]:
full_idt = pd.concat([idt_df1, idt_df2])
full_idt

,Name,Sequence,Concentration,Purification
0,SCO5087_loc_283,cggttggtaggatcgacggcCCGTTCACAGGTCGCGGCGGtatgtc...,25nm,STD
1,SCO5087_loc_522,cggttggtaggatcgacggcGTCGACCTCCCAGTCACGGCtatgtc...,25nm,STD
0,SCO5087_fwd_checking_primer,GACGATTCGGCCCGTG,25nm,STD
1,SCO5087_rev_checking_primer,CAGGGCGTCCAGGC,25nm,STD


## Folder with all the generated I/O

In [25]:
input_files = [
    {"name": "input_genome.gb", "content": genome},
    {"name": "input_plasmid.gb", "content": clean_plasmid}
]

output_files = [
    {"name": "cBEST_w_sgRNAs.gb", "content": sgRNA_vectors}, # LIST OF Dseqrecords
    {"name": "primer_df.csv", "content": checking_primers_df},
    {"name": "full_idt.csv", "content": full_idt},
    {"name": "mutated_sgrna_df.csv", "content": mutated_sgrna_df},
    {"name": "filtered_df.csv", "content": filtered_df},
    {"name": "plasmid_metadata_df.csv", "content": plasmid_metadata_df},

]

input_values = {
    "genes_to_knockout": genes_to_KO,
    "polymerase_settings": {
        "chosen_polymerase": chosen_polymerase,
        "melting_temperature": melting_temperature,
        "primer_concentration": primer_concentration,
        "primer_number_increment": primer_number_increment,
        "flanking_region_number": flanking_region_number
    },
    "filtering_metrics": {
        "gc_upper": gc_upper,
        "gc_lower": gc_lower,
        "off_target_seed": off_target_seed,
        "off_target_upper": off_target_upper,
        "cas_type": cas_type,
        "number_of_sgRNAs_per_group": number_of_sgRNAs_per_group
    },
    "overlapping_sequences": {
        "up_homology": str(up_homology),
        "dw_homology": str(dw_homology)
    }
}


# Paths to Markdown files
markdown_file_paths = [
    "../../protocols/conjugation_protcol.md",
    "../../protocols/single_target_crispr_plasmid_protcol.md"

]

# Data and time
timestamp = datetime.utcnow().isoformat()

# Create project directory structure
project_directory = ProjectDirectory(
    project_name=f"CRISPR_cBEST_workflow_{timestamp}",
    input_files=input_files,
    output_files=output_files,
    input_values=input_values,
    markdown_file_paths=markdown_file_paths
)


# DO You want to save the folder? 
save_zip_folder = False 

if save_zip_folder: 
    # Generate the project directory structure and get the zip content
    zip_content = project_directory.create_directory_structure(create_directories=False)

    # Save the zip file to disk (optional)
    with open("project_structure.zip", "wb") as f:
        f.write(zip_content)